In [1]:
print("heloo")

heloo


In [2]:
import pandas as pd

In [6]:
data = pd.read_csv("C:/Users/mdnai/Downloads/laptop_prices.csv")

In [7]:
data.head()

,Company,Product,TypeName,Inches,Ram,OS,Weight,Price_euros,Screen,ScreenW,...,RetinaDisplay,CPU_company,CPU_freq,CPU_model,PrimaryStorage,SecondaryStorage,PrimaryStorageType,SecondaryStorageType,GPU_company,GPU_model
0,Apple,MacBook Pro,Ultrabook,13.3,8,macOS,1.37,1339.69,Standard,2560,...,Yes,Intel,2.3,Core i5,128,0,SSD,No,Intel,Iris Plus Graphics 640
1,Apple,Macbook Air,Ultrabook,13.3,8,macOS,1.34,898.94,Standard,1440,...,No,Intel,1.8,Core i5,128,0,Flash Storage,No,Intel,HD Graphics 6000
2,HP,250 G6,Notebook,15.6,8,No OS,1.86,575.00,Full HD,1920,...,No,Intel,2.5,Core i5 7200U,256,0,SSD,No,Intel,HD Graphics 620
3,Apple,MacBook Pro,Ultrabook,15.4,16,macOS,1.83,2537.45,Standard,2880,...,Yes,Intel,2.7,Core i7,512,0,SSD,No,AMD,Radeon Pro 455
4,Apple,MacBook Pro,Ultrabook,13.3,8,macOS,1.37,1803.60,Standard,2560,...,Yes,Intel,3.1,Core i5,256,0,SSD,No,Intel,Iris Plus Graphics 650


In [8]:
print(data.to_string())

        Company                                        Product            TypeName  Inches  Ram            OS  Weight  Price_euros       Screen  ScreenW  ScreenH Touchscreen IPSpanel RetinaDisplay CPU_company  CPU_freq                CPU_model  PrimaryStorage  SecondaryStorage PrimaryStorageType SecondaryStorageType GPU_company                GPU_model
0         Apple                                    MacBook Pro           Ultrabook    13.3    8         macOS   1.370      1339.69     Standard     2560     1600          No      Yes           Yes       Intel      2.30                  Core i5             128                 0                SSD                   No       Intel   Iris Plus Graphics 640
1         Apple                                    Macbook Air           Ultrabook    13.3    8         macOS   1.340       898.94     Standard     1440      900          No       No            No       Intel      1.80                  Core i5             128                 0      Flash S

In [9]:
top_brands = data['Company'].value_counts().head(5)
print("Top 5 Laptop Brands by Number of Products Listed:")
print(top_brands)

Top 5 Laptop Brands by Number of Products Listed:
Company
Dell      291
Lenovo    289
HP        268
Asus      152
Acer      101
Name: count, dtype: int64


In [10]:
avg_price_by_brand = data.groupby('Company')['Price_euros'].mean()
print("Average laptop price for each brand:")
print(avg_price_by_brand)

Average laptop price for each brand:
Company
Acer          633.464455
Apple        1564.198571
Asus         1123.829737
Chuwi         314.296667
Dell         1199.225120
Fujitsu       729.000000
Google       1677.666667
HP           1080.314664
Huawei       1424.000000
LG           2099.000000
Lenovo       1093.862215
MSI          1728.908148
Mediacom      295.000000
Microsoft    1612.308333
Razer        3346.142857
Samsung      1413.444444
Toshiba      1267.812500
Vero          217.425000
Xiaomi       1133.462500
Name: Price_euros, dtype: float64


In [11]:
highest_avg_brand = avg_price_by_brand.idxmax()
highest_avg_price = avg_price_by_brand.max()

lowest_avg_brand = avg_price_by_brand.idxmin()
lowest_avg_price = avg_price_by_brand.min()


print(f"\nthe highest average price: {highest_avg_brand} - {highest_avg_price:.2f}")
print(f"the lowest average price: {lowest_avg_brand} - {lowest_avg_price:.2f}")


the highest average price: Razer - 3346.14
the lowest average price: Vero - 217.43


In [12]:
selected_features = ['Price_euros', 'CPU_freq', 'Ram', 'Inches', 'Weight']
numeric_data = data[selected_features]
correlation_matrix = numeric_data.corr()
price_correlation = correlation_matrix['Price_euros'].sort_values(ascending=False)
print("Correlation of features with Price (Price_euros):")
print(price_correlation)

Correlation of features with Price (Price_euros):
Price_euros    1.000000
Ram            0.740287
CPU_freq       0.428847
Weight         0.211883
Inches         0.070091
Name: Price_euros, dtype: float64


In [13]:
most_positive = price_correlation.index[1]
most_negative = price_correlation.index[-1]

print(f"\n Most positively correlated feature with price: {most_positive} - {price_correlation[most_positive]:.2f}")
print(f"Most negatively correlated feature with price: {most_negative} - {price_correlation[most_negative]:.2f}")


 Most positively correlated feature with price: Ram - 0.74
Most negatively correlated feature with price: Inches - 0.07


In [14]:
data['StorageTotal'] = data['PrimaryStorage'] + data['SecondaryStorage']
print(data[['PrimaryStorage', 'SecondaryStorage', 'StorageTotal']].head())

   PrimaryStorage  SecondaryStorage  StorageTotal
0             128                 0           128
1             128                 0           128
2             256                 0           256
3             512                 0           512
4             256                 0           256


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor

In [16]:

features = ['Ram', 'Inches', 'CPU_freq', 'PrimaryStorage', 'GPU_company']
target = 'Price_euros'

data = data.dropna(subset=features + [target])
X = data[features]
y = data[target]


categorical_cols = ['GPU_company']
numerical_cols = ['Ram', 'Inches', 'CPU_freq', 'PrimaryStorage']


preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ('num', 'passthrough', numerical_cols)
])


models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(random_state=42),
    'DecisionTree': DecisionTreeRegressor(random_state=42)
}

best_model = None
best_score = -1

# Train and evaluate each model
for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    score = r2_score(y_test, y_pred)
    print(f'{name} R² Score: {score:.3f}')

    if score > best_score:
        best_score = score
        best_model = name

print(f'\nBest Regression Model: {best_model} (R² = {best_score:.3f})')

LinearRegression R² Score: 0.637
RandomForest R² Score: 0.802
DecisionTree R² Score: 0.792

Best Regression Model: RandomForest (R² = 0.802)


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.inspection import permutation_importance

In [19]:

features = ['Ram', 'Inches', 'CPU_freq', 'PrimaryStorage', 'GPU_company']
target = 'Price_euros'

data = data.dropna(subset=features + [target])
X = data[features]
y = data[target]


categorical_cols = ['GPU_company']
numerical_cols = ['Ram', 'Inches', 'CPU_freq', 'PrimaryStorage']


preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ('num', 'passthrough', numerical_cols)
])


models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(random_state=42),
    'DecisionTree': DecisionTreeRegressor(random_state=42)
}

best_model = None
best_score = -1


for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    score = r2_score(y_test, y_pred)
    print(f'{name} R² Score: {score:.3f}')


    if score > best_score:
        best_score = score
        best_model = name

print(f'\nBest Regression Model: {best_model} (R² = {best_score:.3f})')


features = ['Inches', 'Ram', 'PrimaryStorage', 'Weight']
target = 'TypeName'


data = data.dropna(subset=features + [target])

X = data[features]
y = data[target]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42)
}

best_model = None
best_score = 0
best_model_name = ""

for name, model in models.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Scale features
        ('classifier', model)
    ])

  
    pipeline.fit(X_train, y_train)

   
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy:.3f}")

    if accuracy > best_score:
        best_score = accuracy
        best_model = pipeline
        best_model_name = name

print(f"\nBest Classification Model: {best_model_name} (Accuracy = {best_score:.3f})")

if best_model_name in ['Random Forest', 'Decision Tree']:
    importance = best_model.named_steps['classifier'].feature_importances_
    feature_importance = pd.Series(importance, index=features).sort_values(ascending=False)

    print("\nFeature Importance:")
    print(feature_importance)
else:

    result = permutation_importance(best_model, X_test, y_test, n_repeats=10, random_state=42)
    importance = pd.Series(result.importances_mean, index=features).sort_values(ascending=False)

    print("\nFeature Importance (Permutation Based):")
    print(importance)

LinearRegression R² Score: 0.637
RandomForest R² Score: 0.802
DecisionTree R² Score: 0.792

Best Regression Model: RandomForest (R² = 0.802)
Logistic Regression Accuracy: 0.685
Random Forest Accuracy: 0.713
Decision Tree Accuracy: 0.693

Best Classification Model: Random Forest (Accuracy = 0.713)

Feature Importance:
Weight            0.529745
Inches            0.187945
Ram               0.177987
PrimaryStorage    0.104323
dtype: float64
